In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/dataset/Animal Detection.v1i.yolov8.zip'  # change path
extract_path = '/content/animal_dataset/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)


Dataset extracted to: /content/animal_dataset/


In [5]:
pip install  ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.5 MB/s eta 0:00:00


In [6]:
# YOLOv8 and PyTorch
from ultralytics import YOLO
import torch

# Data handling and image processing
import cv2
import numpy as np
import os
import yaml

# GUI
import gradio as gr

# Training utilities
from torch.optim import Adam


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# YOLOv8 already supports augmentation during training
# But for GUI inference, we need to auto-resize imported images

def preprocess_image(img, target_size=640):
    """
    Resize image maintaining aspect ratio and pad to target_size.
    """
    h, w = img.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h))
    padded = np.full((target_size, target_size, 3), 114, dtype=np.uint8)
    padded[:new_h, :new_w] = resized
    return padded


In [8]:
# YOLOv8 uses built-in optimizers, but can specify custom one if needed
optimizer_params = {
    'lr0': 0.01,   # initial learning rate
    'momentum': 0.937,
    'weight_decay': 0.0005
}


In [9]:
# Load pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train
model.train(
    data='/content/animal_dataset/data.yaml',  # path to your Roboflow data.yaml
    epochs=20,                                 # fine-tune 15-20 epochs
    imgsz=640,
    batch=16,
    patience=5,                                # early stopping patience
    optimizer='Adam'
)

# Save best model
best_model_path = '/content/best_yolov8n_animal.pt'
model.save(best_model_path)
print("Best model saved at:", best_model_path)


Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/animal_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrai

Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,013,188 parameters, 3,013,172 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1672.1±1062.0 MB/s, size: 54.1 KB)


train: Scanning /content/animal_dataset/train/labels... 5572 images, 40 backgrounds, 0 corrupt: 100%|██████████| 5572/5572 [00:03<00:00, 1725.82it/s]

...

                   all       1204       1530      0.717      0.573       0.63      0.449

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      2.92G     0.9336      1.204       1.37          8        640: 100%|██████████| 349/349 [01:30<00:00,  3.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:08<00:00,  4.41it/s]


                   all       1204       1530      0.745       0.58      0.645      0.466

20 epochs completed in 0.576 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:11<00:00,  3.18it/s]


                   all       1204       1530      0.745       0.58      0.645      0.466
                  Bear        220        281      0.853      0.847      0.899       0.75
                  Deer        117        158      0.641      0.684      0.732      0.419
                   Fox        100        120      0.492      0.458      0.504      0.361
                  Goat        142        265      0.639      0.461      0.551      0.352
                  Kurt        215        265      0.795      0.589      0.702      0.434
              Wild-Pig        151        166      0.802      0.879        0.9      0.732
           Wild-Piglet         10         16          1          0      0.102     0.0781
                badger         40         41      0.818      0.547      0.661      0.458
                coyote        207        218      0.666      0.751      0.756      0.608
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect

In [10]:

metrics = model.val()
print(metrics)


Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2226.6±519.9 MB/s, size: 79.8 KB)


val: Scanning /content/animal_dataset/valid/labels.cache... 1204 images, 12 backgrounds, 0 corrupt: 100%|██████████| 1204/1204 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 504, len(boxes) = 1530. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:12<00:00,  6.09it/s]


                   all       1204       1530      0.745      0.579      0.645      0.466
                  Bear        220        281      0.852      0.847      0.899      0.748
                  Deer        117        158      0.643      0.684      0.732      0.419
                   Fox        100        120      0.492      0.458      0.503       0.36
                  Goat        142        265      0.639      0.461      0.551      0.351
                  Kurt        215        265      0.795      0.586      0.702      0.435
              Wild-Pig        151        166      0.802      0.879        0.9      0.734
           Wild-Piglet         10         16          1          0      0.102      0.078
                badger         40         41      0.818      0.547      0.661      0.464
                coyote        207        218      0.667      0.752      0.756      0.609
Speed: 0.9ms preprocess, 3.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect

In [31]:
import cv2
import numpy as np
import gradio as gr
from ultralytics import YOLO
import tempfile

# Load fine-tuned YOLOv8 model
best_model_path = '/content/drive/MyDrive/dataset/best_yolov8n_animal.pt'
model = YOLO(best_model_path)

# List of carnivore classes
carnivores = ['lion', 'tiger', 'wolf', 'fox']

# Preprocess image (PIL → BGR NumPy array)
def preprocess_image(img):
    img = np.array(img)
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

# Detect animals in image
def detect_animals_image(img):
    img = preprocess_image(img)
    results = model.predict(img, imgsz=640, conf=0.25)
    annotated_img = img.copy()
    carn_count = 0

    if results[0].boxes is not None:
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            cls_name = model.names[cls_id].strip().lower()
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            if cls_name in [c.lower() for c in carnivores]:
                color = (0, 0, 255)   # Red for carnivores
                carn_count += 1
            else:
                color = (0, 255, 0)   # Green for non-carnivores

            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_img, cls_name, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Convert back to RGB for correct display
    annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    message = f"Detected {carn_count} carnivorous animals." if carn_count > 0 else "No carnivorous animals detected."
    return annotated_img, message

# Detect animals in video
def detect_animals_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    tmp_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    out = None
    total_carn_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, imgsz=640, conf=0.25, verbose=False)
        carn_count = 0
        annotated_frame = frame.copy()

        if results[0].boxes is not None:
            for box in results[0].boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id].strip().lower()
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                if cls_name in [c.lower() for c in carnivores]:
                    color = (0, 0, 255)   # Red for carnivores
                    carn_count += 1
                else:
                    color = (0, 255, 0)   # Green for others

                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(annotated_frame, cls_name, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        total_carn_count += carn_count

        # Convert frame to RGB before saving
        annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

        if out is None:
            h, w = annotated_frame.shape[:2]
            out = cv2.VideoWriter(tmp_out.name, fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

        out.write(annotated_frame)

    cap.release()
    out.release()
    message = f"Detected {total_carn_count} carnivorous animals in video." if total_carn_count > 0 else "No carnivorous animals detected in video."
    return tmp_out.name, message

# Gradio app with tabs (Image + Video)
with gr.Blocks() as demo:
    gr.Markdown("## Animal Detection with Carnivore Highlighting\nPlease upload a clear image/video before detection.")

    with gr.Tab("Image"):
        img_in = gr.Image(type="numpy", label="Upload Image")
        img_out = gr.Image(type="numpy", label="Detected Image")
        msg_out = gr.Textbox(label="Detection Result")
        img_btn = gr.Button("Detect Animals")
        img_btn.click(fn=detect_animals_image, inputs=img_in, outputs=[img_out, msg_out])

    with gr.Tab("Video"):
        vid_in = gr.Video(label="Upload Video")
        vid_out = gr.Video(label="Detected Video")
        vid_msg = gr.Textbox(label="Detection Result")
        vid_btn = gr.Button("Detect Animals in Video")
        vid_btn.click(fn=detect_animals_video, inputs=vid_in, outputs=[vid_out, vid_msg])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://850e02bbd93887d19e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [38]:
# Save the current model weights
model.save("my_custom_weights.pt")
